# Palabras Relacionadas - Dataset

En este notebook se explica en detalle el dataset y el procesamiento que requirió para dejarlo listo para la siguiente etapa del proyecto.

En primer lugar, se buscó y seleccionó múltiples textos en formato PDF pertenencientes a distintas materias de la carrera Ingeniería en Sistemas de Información, Universidad Tecnológica Nacional Facultad Regional Mendoza.

Estos, fueron cargados en la carpeta /data/raw.

Se siguió como convención para los nombres el número del año de la materia, seguido de un guión, una abreviatura del nombre de la materia, otro guión y el nombre original del material.

Utilizando algunas librerías de python, se convirtió cada archivo PDF en un archivo txt con su contenido, en /data/plain

In [ ]:
!pip install pypdf

In [1]:
from pypdf import PdfReader
import os
from os import listdir
from os.path import isfile, join

# Seteo el path al root del proyecto
dev_folder = 'dev'
folders = os.getcwd().split('/')
if (len(folders) == 1):
    folders = folders[0].split('\\')
if(folders[-1] == dev_folder):
    os.chdir('../')
print(os.getcwd())

c:\Users\saoga\OneDrive\Escritorio\Repos\TPI-RNP-Palabras-Relacionadas-ISI


In [2]:
raw_path = "./data/raw"
plain_path = "./data/plain"

# Arreglo con los nombres de los archivos PDF
raw_files = [os.path.splitext(f)[0] for f in listdir(raw_path) if isfile(join(raw_path, f))]

In [ ]:
metrics = {}
for f in raw_files:
    print("\033[94mConvirtiendo archivo: " + f + "\033[0m")

    sf = f.split("-")
    anio = sf[0].strip()
    materia = sf[1].strip()
    if (not anio in metrics):
        metrics[anio] = {}

    if (not materia in metrics[anio]):
        metrics[anio][materia] = 0

    metrics[anio][materia] += os.path.getsize(raw_path + "/" + f + ".pdf")

    reader = PdfReader(raw_path + "/" + f + ".pdf")
    for p in reader.pages:
        with open(plain_path + "/" + f + ".txt", "ab") as t:
            t.write(p.extract_text(
                extraction_mode="plain",
                layout_mode_space_vertically=False).encode("utf-8"))

print("\033[92m")
for a, materias in metrics.items():
    print("Año " + a + ":")
    for m, tamano in materias.items():
        print("\tMateria: " + m + " - " + str(round(tamano/1000000,2)) +"MB")
print("\033[0m")

Convirtiendo archivo: 1 - AC - LibroArquitecturadeComputadorasSantiagoPerez090321
Convirtiendo archivo: 1 - AyED - cpp según yo pero en pedo
Convirtiendo archivo: 1 - AyED - cpp según yo
Convirtiendo archivo: 1 - AyED - Unidad3 (7929)
Convirtiendo archivo: 1 - AyED - Unidad4 (7930)
Convirtiendo archivo: 1 - AyED - Unidades 1 y 2 (cód. fotoc. 7928)
Convirtiendo archivo: 1 - MD - Matemáticas discretas by Ramóne Espinosa Armenta (z-lib.org)
Convirtiendo archivo: 1 - SyO - 01 Evolucion de las estructuras
Convirtiendo archivo: 1 - SyO - 02 Gestion por procesos UNCuyo
Convirtiendo archivo: 1 - SyO - 03 Gestion por procesos, indicaroes y estandares para unidades de informacion - Cap 1 y 2
Convirtiendo archivo: 1 - SyO - 04 gestion-por-procesos
Convirtiendo archivo: 1 - SyO - 05 Arquitectura_empresarial_que_es_y_para_q
Convirtiendo archivo: 1 - SyO - 1)La Información en la Empresa
Convirtiendo archivo: 1 - SyO - 2)Recopilacion de la informacion
Convirtiendo archivo: 1 - SyO - Analisis FODA
Con

A continuación, se tokenizó los archivos planos txt, generando un archivo txt nuevo donde cada línea es un token.

Esto se logró gracias a Spacy.

In [ ]:
!pip install spacy
!python -m spacy download es_core_news_sm

In [8]:
import spacy

tokens_path = "./data/tokens"

esp = spacy.load("es_core_news_sm")

In [ ]:
def is_clean_token(token):
    return not (
        token.is_punct or
        token.is_space or
        token.is_stop or
        len(token.text) == 1)

for f in raw_files:
    print("\033[94mTokenizando archivo: " + f + "\033[0m")

    with open(plain_path + "/" + f + ".txt", "rb") as pf:
        txt = pf.read().decode("utf-8")
        tokens = esp.tokenizer(txt)
        with open(tokens_path + "/" + f + ".txt", "wb") as tf:
            for token in tokens:
                if (is_clean_token(token)):
                    tf.write((token.text + "\n").encode("utf-8"))

Tokenizando archivo: 1 - AC - LibroArquitecturadeComputadorasSantiagoPerez090321
Tokenizando archivo: 1 - AyED - cpp según yo pero en pedo
Tokenizando archivo: 1 - AyED - cpp según yo
Tokenizando archivo: 1 - AyED - Unidad3 (7929)
Tokenizando archivo: 1 - AyED - Unidad4 (7930)
Tokenizando archivo: 1 - AyED - Unidades 1 y 2 (cód. fotoc. 7928)
Tokenizando archivo: 1 - MD - Matemáticas discretas by Ramóne Espinosa Armenta (z-lib.org)
Tokenizando archivo: 1 - SyO - 01 Evolucion de las estructuras
Tokenizando archivo: 1 - SyO - 02 Gestion por procesos UNCuyo
Tokenizando archivo: 1 - SyO - 03 Gestion por procesos, indicaroes y estandares para unidades de informacion - Cap 1 y 2
Tokenizando archivo: 1 - SyO - 04 gestion-por-procesos
Tokenizando archivo: 1 - SyO - 05 Arquitectura_empresarial_que_es_y_para_q
Tokenizando archivo: 1 - SyO - 1)La Información en la Empresa
Tokenizando archivo: 1 - SyO - 2)Recopilacion de la informacion
Tokenizando archivo: 1 - SyO - Analisis FODA
Tokenizando archiv

Una vez que se tienen los archivos con los tokens, deseamos detectar conceptos dentro de ellos. Estos conceptos formarán luego nuestro vocabulario.

Se utilizará un algoritmo de ventana deslizante para esto.

In [3]:
from itertools import combinations
import re

window_size = 4

In [ ]:
related_table = {}

banned = ["capítulo", "página", "figura", "cap", "ejemplo", "catedra", "mendoza", "argentina", "muñoz", "facchini", "cesari", "xsd", "infoleg"]

for f in raw_files:
    print("\033[94mDetectando conceptos en archivo: " + f + "\033[0m")

    tokens = []

    with open(tokens_path + "/" + f + ".txt", "rb") as tf:
        tokens = tf.read().decode("utf-8").split("\n")

    for i in range(len(tokens) - window_size):
        window = tokens[i:i+window_size]

        def get_subarrays(arr):
            result = []
            n = len(arr)
            for r in range(1, n+1):  # sizes from 1 to n
                for indices in combinations(range(n), r):
                    subarray = [arr[i] for i in indices]
                    result.append(subarray)
            return result
        
        arrays = get_subarrays(window[1:])

        # arrays.insert(0, []) # Permite formar conceptos de una sola palabra

        for arr in arrays:
            arr.insert(0, window[0])
            arr = [s.lower() for s in arr]

            if any(re.search(r'(^[0-9\.\,]+$)|(-$)|(^.\.$)', s) for s in arr):
                continue

            if len(arr) != len(set(arr)):
                continue

            if any(ban in arr for ban in banned):
                continue

            arr.sort()
            
            t = tuple(arr)
            if (not t in related_table):
                related_table[t] = 0
            related_table[t] += 1

print("\033[92mCantidad de conceptos candidatos:" + str(len(related_table)) + "\033[0m")

Detectando conceptos en archivo: 1 - AC - LibroArquitecturadeComputadorasSantiagoPerez090321
Detectando conceptos en archivo: 1 - AyED - cpp según yo pero en pedo
Detectando conceptos en archivo: 1 - AyED - cpp según yo
Detectando conceptos en archivo: 1 - AyED - Unidad3 (7929)
Detectando conceptos en archivo: 1 - AyED - Unidad4 (7930)
Detectando conceptos en archivo: 1 - AyED - Unidades 1 y 2 (cód. fotoc. 7928)
Detectando conceptos en archivo: 1 - MD - Matemáticas discretas by Ramóne Espinosa Armenta (z-lib.org)
Detectando conceptos en archivo: 1 - SyO - 01 Evolucion de las estructuras
Detectando conceptos en archivo: 1 - SyO - 02 Gestion por procesos UNCuyo
Detectando conceptos en archivo: 1 - SyO - 03 Gestion por procesos, indicaroes y estandares para unidades de informacion - Cap 1 y 2
Detectando conceptos en archivo: 1 - SyO - 04 gestion-por-procesos
Detectando conceptos en archivo: 1 - SyO - 05 Arquitectura_empresarial_que_es_y_para_q
Detectando conceptos en archivo: 1 - SyO - 1)

En el diccionario related_table está contenido cuántas veces aparecieron ciertos tokens en la ventana.

A continuación, en vocabulary, se almacenarán solo los que superen una frecuencia mínima y no otra máxima.

In [60]:
min_freq = 50
max_freq = 100

vocabulary = []

for tokens, freq in related_table.items():
    if (freq >= min_freq and freq <= max_freq):
        vocabulary.append(tokens)

concepts_file = "./data/vocabulary.txt"

with open(concepts_file, "wb") as cf:
    for concept in vocabulary:
        cf.write((str(concept) + "\n").encode("utf-8"))

print("\033[92mTamaño del vocabulario:" + str(len(vocabulary)) + "\033[0m")

Tamaño del vocabulario:2726


En este punto, ya hemos seleccionado conjuntos de tokens que suelen aparecer cerca.

Estos conjuntos serán los conceptos, y pasarán a formar nuestro vocabulario.

Ahora, se debe tokenizar nuevamente los textos planos, utilizando los conceptos.

Para esto, se recorrerá cada archivo de /data/plain mediante una ventana deslizante del mismo tamaño utilizado para detectar conceptos, separando en palabras siempre y cuando no se encuentre dentro de la ventana las palabras de un concepto.

Se generarán nuevos tokens, siendo estos numéricos (/data/tokens_num). LLos positivos (o 0) corresponden al índice de un concepto en el vocabulario, mientras que los negativos indican la cantidad de tokens no reconocidos (\<unk\>). Esto se realizó de esta forma para ahorrar espacio y tiempo de procesamiento.

Si en una ventana se detectara más de un concepto, se agregarán todos los que se encuentre. Debido al procesamiento que se realizará más adelante, no debería importar el orden.

In [7]:
import ast

concepts_file = "./data/vocabulary.txt"
vocabulary = []

with open(concepts_file, "rb") as cf:
    lines = cf.read().decode("utf-8").split("\n")[:-1]
    vocabulary = [ast.literal_eval(l) for l in lines]

print("\033[92mTamaño del vocabulario:" + str(len(vocabulary)) + "\033[0m")

tokens_conceptos_path = "./data/tokens_conceptos"

Tamaño del vocabulario:2726


In [ ]:
metrics_2 = {}

window_size_concept_tokenization = window_size * 3

for f in raw_files:
    print("\033[94mTokenizando archivo por conceptos: " + f + "\033[0m")
    metrics_2[f] = {}
    found_concepts = 0

    recent_concepts = {}

    with open(plain_path + "/" + f + ".txt", "rb") as pf:
        txt = pf.read().decode("utf-8")
        tokens = [t.text for t in esp.tokenizer(txt)]

        metrics_2[f]["tokens"] = len(tokens)

        with open(tokens_conceptos_path + "/" + f + ".txt", "wb") as tnf:

            unks = 0

            for i in range(len(tokens) - window_size_concept_tokenization):
                window = tokens[i:i+window_size_concept_tokenization]
                for k, v in recent_concepts.items():
                    if v > 0:
                        recent_concepts[k] -= 1

                unks += 1
            
                for ix, concept in enumerate(vocabulary):
                    if all(word in window for word in concept) and (not ix in recent_concepts or recent_concepts[ix] == 0):
                        tnf.write(("-" + str((unks-1)) + " " + str(ix) + " ").encode("utf-8"))
                        unks = 0
                        recent_concepts[ix] = window_size_concept_tokenization
                        found_concepts += 1

                    
    metrics_2[f]["concepts"] = found_concepts

okenizando archivo por conceptos: 3 - AP - AdProy_2_Trabajo en Equipo_2022
okenizando archivo por conceptos: 3 - AP - respuestas
okenizando archivo por conceptos: 3 - BD - caselli_manual-de-base-de-datos-2019
okenizando archivo por conceptos: 3 - BD - Guía 1
okenizando archivo por conceptos: 3 - BD - Guía 2
okenizando archivo por conceptos: 3 - CD - capitulo2
okenizando archivo por conceptos: 3 - CD - Comunicaciones y Redes de Computadores,7ma Edición - William Stallings
okenizando archivo por conceptos: 3 - CD - sistemas-de-comunicaciones-electronicas-tomasi-4ta-edicion
okenizando archivo por conceptos: 3 - DS - Actor. Definicion. Clasificacion (1)
okenizando archivo por conceptos: 3 - DS - Eje 1. Metodología y conceptos teóricos aplicados
okenizando archivo por conceptos: 3 - DS - Libro UML y Patrones - Larman
okenizando archivo por conceptos: 3 - DS - MerFNConceptos
okenizando archivo por conceptos: 4 - AS - Analisis PEST
okenizando archivo por conceptos: 4 - AS - Backups_raids
oken

In [10]:
print("\033[92m")
for archivo, item in metrics_2.items():
    print("Archivo " + archivo + ":")
    for nombre, valor in item.items():
        print("\t" + nombre + ": " + str(valor))
print("\033[0m")


Archivo 3 - AP - AdProy_2_Trabajo en Equipo_2022:
	tokens: 27662
	concepts: 275
Archivo 3 - AP - respuestas:
	tokens: 0
	concepts: 0
Archivo 3 - BD - caselli_manual-de-base-de-datos-2019:
	tokens: 28732
	concepts: 388
Archivo 3 - BD - Guía 1:
	tokens: 11365
	concepts: 390
Archivo 3 - BD - Guía 2:
	tokens: 9548
	concepts: 143
Archivo 3 - CD - capitulo2:
	tokens: 10412
	concepts: 236
Archivo 3 - CD - Comunicaciones y Redes de Computadores,7ma Edición - William Stallings:
	tokens: 455695
	concepts: 17102
Archivo 3 - CD - sistemas-de-comunicaciones-electronicas-tomasi-4ta-edicion:
	tokens: 530998
	concepts: 19677
Archivo 3 - DS - Actor. Definicion. Clasificacion (1):
	tokens: 1490
	concepts: 33
Archivo 3 - DS - Eje 1. Metodología y conceptos teóricos aplicados:
	tokens: 4415
	concepts: 41
Archivo 3 - DS - Libro UML y Patrones - Larman:
	tokens: 253057
	concepts: 3868
Archivo 3 - DS - MerFNConceptos:
	tokens: 2342
	concepts: 42
Archivo 4 - AS - Analisis PEST:
	tokens: 1004
	concepts: 4
Arc

A partir de los tokens numéricos, se iterará por cada secuencia de token con un nuevo tamaño de ventana, mayor, tratando de distinguir conceptos relacionados.

Esta ventana se centrará en cada token (no -1), almacenando en un diccionario el token central, los tokens en el contexto y ejemplos negativos (para evitar que la red neuronal, al entrenar, aprenda que todos los tokens siempre están relacionados).

Para el muestreo negativo, primero se debe calcular la frecuencia relativa de cada token:

In [45]:
freq_abs = {}
for file in raw_files:
    with open(f"{tokens_conceptos_path}/{file}.txt", "rb") as pf:
        txt = pf.read().decode("utf-8")

        nums = txt.split(" ") # Lista con cada número en el archivo
        for num in nums:
            if (num == "" or num[0] == "-"):
                continue

            token_ix = int(num)
            if not token_ix in freq_abs:
                freq_abs[token_ix] = 0

            freq_abs[token_ix] += 1

total_tokens = 0
for token, freq in freq_abs.items():
    total_tokens += freq

freq_rel = {}
for token, freq in freq_abs.items():
    freq_rel[token] = freq/total_tokens
        

A su vez, se definirá una clase auxiliar:

In [48]:
import random

class RandomGenerator:
  """Randomly draw among {1, ..., n} according to n sampling weights."""
  def __init__(self, sampling_weights):
    # Exclude
    self.population = list(range(1, len(sampling_weights) + 1))
    self.sampling_weights = sampling_weights
    self.candidates = []
    self.i = 0

  def draw(self):
    if self.i == len(self.candidates):
      # Cache `k` random sampling results
      self.candidates = random.choices(
          self.population, self.sampling_weights, k=10000)
      self.i = 0
    self.i += 1
    return self.candidates[self.i - 1]

Hecho esto, podemos finalmente armar nuestro dataset. El mismo retornará (mediante get_item()) un centro, su contexto y sus ejemplos negativos.

In [ ]:
!pip install torch

In [ ]:
import torch
import pickle

class ISIDataset(torch.utils.data.Dataset):

    def __init__(self):
        self.data = []

    def create(self, files, window_size, tokens_conceptos_path, K=5, load_file = "./data/dataset.pkl"):

        window_radius = window_size // 2

        for file in files:
            print("\033[94mArmando dataset con: " + file + "\033[0m")

            with open(f"{tokens_conceptos_path}/{file}.txt", "rb") as pf:
                txt = pf.read().decode("utf-8")

                nums = txt.split(" ") # Lista con cada número en el archivo

                for ix, num in enumerate(nums):
                    if (num == "" or num[0] == "-"):
                        continue

                    token_ix = int(num)

                    context = []
                    negatives = []
                    
                    c = window_radius
                    i = 1
                    while c >= 0: # Buscar conceptos hacia atrás
                        if (ix - i < 0): # Si se acabó el archivo, dejar de buscar
                            break
                        
                        val = nums[ix - i]
                        if (val == ""): 
                            val = "-0"
                        if (val[0] == "-"):
                            val = val.lstrip("-")
                            unks = int(val)
                            c = c - unks
                        else:
                            concept = int(val)
                            context.append(concept)
                            c = c - 1
                        i = i + 1

                        
                    c = window_radius
                    i = 1
                    l = len(nums)
                    while c >= 0: # Buscar conceptos hacia adelante
                        if (ix + i < l): # Si se acabó el archivo, dejar de buscar
                            break
                        
                        val = nums[ix + i]
                        if (val == ""): 
                            val = "-0"
                        if (val[0] == "-"):
                            val = val.lstrip("-")
                            unks = int(val)
                            c = c - unks
                        else:
                            concept = int(val)
                            context.append(concept)
                            c = c - 1
                        i = i + 1
                    
                    sampling_weights = [freq_rel[concept]**0.75 if concept in freq_rel else 0 for concept in range(0, len(vocabulary))]
                    generator = RandomGenerator(sampling_weights)
                    while len(negatives) < len(context) * K:
                        neg = generator.draw()
                        if neg not in context:
                            negatives.append(neg)

                    if (len(context) > 0):
                        self.data.append({
                            "center": token_ix,
                            "context": context,
                            "negatives": negatives
                        })
        
        with open(load_file, "wb") as lf:
            pickle.dump(self.data, lf)


    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return len(self.data)
    
    def load(self, load_file):
        with open(load_file, "rb") as lf:
            self.data = pickle.load(lf)



Para poder probar distintas alternativas, se generó distintos datasets (almacenados en ./data/datasets), con distintos tamaños de ventana y cantidad de ejemplos negativos (K).

In [ ]:
full_window_sizes = [50, 100, 200, 300, 500, 1000]
Ks = [1, 2, 3, 5]

datasets_path = "./data/datasets"

for ws in full_window_sizes:
    for K in Ks:
        dataset = ISIDataset()
        dataset.create(raw_files, ws, tokens_conceptos_path, K, datasets_path + "/" + str(ws) + "-" + str(K) + ".pkl")

Armando dataset con: 1 - AC - LibroArquitecturadeComputadorasSantiagoPerez090321
Armando dataset con: 1 - AyED - cpp según yo pero en pedo
Armando dataset con: 1 - AyED - cpp según yo
Armando dataset con: 1 - AyED - Unidad3 (7929)
Armando dataset con: 1 - AyED - Unidad4 (7930)
Armando dataset con: 1 - AyED - Unidades 1 y 2 (cód. fotoc. 7928)
Armando dataset con: 1 - MD - Matemáticas discretas by Ramóne Espinosa Armenta (z-lib.org)
Armando dataset con: 1 - SyO - 01 Evolucion de las estructuras
Armando dataset con: 1 - SyO - 02 Gestion por procesos UNCuyo
Armando dataset con: 1 - SyO - 03 Gestion por procesos, indicaroes y estandares para unidades de informacion - Cap 1 y 2
Armando dataset con: 1 - SyO - 04 gestion-por-procesos
Armando dataset con: 1 - SyO - 05 Arquitectura_empresarial_que_es_y_para_q
Armando dataset con: 1 - SyO - 1)La Información en la Empresa
Armando dataset con: 1 - SyO - 2)Recopilacion de la informacion
Armando dataset con: 1 - SyO - Analisis FODA
Armando dataset co

La siguiente celda muestra un ejemplo de cómo cargar uno de los datasets generados.

In [194]:
dataset = ISIDataset()

dataset.load(datasets_path + "/50-1.pkl")

print(len(dataset))

85360


A partir de este Dataset se generará un DataLoader.

Un requisito que debe cumplir es que se modifique los lotes para que estos tengan la misma longitud (cosa que no pasa debido a las diferentes cantidades de conceptos en el contexto de cada concepto central y, por ende de ejemplos negativos). Para solventar esto se utiliza la función `collate_batch`.

In [195]:
def collate_batch(data):
  max_len = max(len(d["context"]) + len(d["negatives"]) for d in data)
  centers, contexts_negatives, masks, labels = [], [], [], []
  for d in data:
    center = d["center"]
    context = d["context"]
    negative = d["negatives"]
    centers += [center]
    cur_len = len(context) + len(negative)
    contexts_negatives += [context + negative + [len(vocabulary)] * (max_len - cur_len)]
    masks += [[1] * cur_len + [0] * (max_len - cur_len)]
    labels += [[1] * len(context) + [0] * (max_len - len(context))]
  return (torch.tensor(centers).reshape((-1, 1)), torch.tensor(
        contexts_negatives), torch.tensor(masks), torch.tensor(labels))

In [196]:
batch_size = 512
dataloader = torch.utils.data.DataLoader(dataset, batch_size, shuffle=True, collate_fn=collate_batch)

A continuación, se armará la estructura de la red neuronal mediante skipgram, utilizando capas Embedding de pytorch.

In [198]:
from torch import nn

class SkipGram(nn.Module):
    def __init__(self, vocabulary, embed_size):
        super().__init__()
        self.central_embedding = nn.Embedding(num_embeddings=len(vocabulary)+1,
                                embedding_dim=embed_size, padding_idx=len(vocabulary))
        self.context_embedding = nn.Embedding(num_embeddings=len(vocabulary)+1,
                                embedding_dim=embed_size, padding_idx=len(vocabulary))

    def forward(self, center, contexts_and_negatives):
        v = self.central_embedding(center)
        u = self.context_embedding(contexts_and_negatives)
        pred = torch.bmm(v, u.permute(0, 2, 1))
        return pred

Como función de pérdida, se utilizará entropía cruzada binaria (Sigmoidea). Esto es así pues requerimos clasificar dos conceptos según si están o no relacionado.

In [199]:
class SigmoidBCELoss(nn.Module):
    # Binary cross-entropy loss with masking
    def __init__(self):
        super().__init__()

    def forward(self, inputs, target, mask=None):
        out = nn.functional.binary_cross_entropy_with_logits(
            inputs, target, weight=mask, reduction="none")
        return out.mean(dim=1)

loss = SigmoidBCELoss()

Se optó por abarcar todo el entrenamiento en una misma función. La misma incluye la inicialización de variables y el ciclo de entrenamiento en sí.

In [200]:
import time
def train(net, data_iter, lr, num_epochs, device):
    def init_weights(module):
        if type(module) == nn.Embedding:
            nn.init.xavier_uniform_(module.weight)
    net.apply(init_weights)
    net = net.to(device)
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)
    L = 0
    N = 0
    for epoch in range(num_epochs):
        start, num_batches = time.time(), len(data_iter)
        for i, batch in enumerate(data_iter):
            optimizer.zero_grad()
            center, context_negative, mask, label = [
                data.to(device) for data in batch]

            pred = net(center, context_negative)
            l = (loss(pred.reshape(label.shape).float(), label.float(), mask)
                     / mask.sum(axis=1) * mask.shape[1])
            l.sum().backward()
            optimizer.step()
            L += l.sum()
            N += l.numel()
        print(f'loss {L / N:.3f}, '
          f'{N / (time.time() - start):.1f} tokens/sec on {str(device)}')

Se generó una función auxiliar para el entrenamiento por medio de GPU, en caso de estar disponible.

In [201]:
def try_gpu(i=0):
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

Finalmente, se realizó el entrenamiento. Esto se realizó con cada dataset, almacenando los parámetros resultantes de cada uno (variando tamaños de los embeddings y de lote) en ./data/models.

In [205]:
lr, num_epochs = 0.002, 20

embed_sizes = [256, 512]
batch_sizes = [256, 512]

models_path = "./data/models"

raw_files = [os.path.splitext(f)[0] for f in listdir(datasets_path) if isfile(join(datasets_path, f))]

for f in raw_files:
    ds = ISIDataset()
    ds.load(datasets_path + "/" + f + ".pkl")
    for bs in batch_sizes:
        for es in embed_sizes:
            print("\033[94mEntrenando modelo " + f + "-" + str(bs) + "-" + str(es) + "\033[0m")
            dl = torch.utils.data.DataLoader(ds, bs, shuffle=True, collate_fn=collate_batch)
            isinet = SkipGram(vocabulary, es)
            train(isinet, dl, lr, num_epochs, try_gpu())
            torch.save(isinet.state_dict(), models_path + "/" + f + "-" + str(bs) + "-" + str(es) + ".pt")

Entrenando modelo 100-1-256-256
loss 0.624, 25517.7 tokens/sec on cpu
loss 0.505, 56303.9 tokens/sec on cpu
loss 0.424, 68503.3 tokens/sec on cpu
loss 0.368, 90348.5 tokens/sec on cpu
loss 0.326, 120461.2 tokens/sec on cpu
loss 0.293, 125497.5 tokens/sec on cpu
loss 0.266, 146860.6 tokens/sec on cpu
loss 0.244, 171941.1 tokens/sec on cpu
loss 0.226, 187527.1 tokens/sec on cpu
loss 0.210, 203286.1 tokens/sec on cpu
loss 0.197, 225375.2 tokens/sec on cpu
loss 0.186, 230978.7 tokens/sec on cpu
loss 0.176, 244238.8 tokens/sec on cpu
loss 0.168, 291088.7 tokens/sec on cpu
loss 0.160, 290201.6 tokens/sec on cpu
loss 0.154, 296427.8 tokens/sec on cpu
loss 0.148, 212445.5 tokens/sec on cpu
loss 0.143, 328267.5 tokens/sec on cpu
loss 0.138, 385471.2 tokens/sec on cpu
loss 0.134, 369284.6 tokens/sec on cpu
Entrenando modelo 100-1-256-512
loss 0.601, 13474.5 tokens/sec on cpu
loss 0.466, 28138.2 tokens/sec on cpu


KeyboardInterrupt: 

Para verificar la funcionalidad final que buscamos en el proyecto, se planteó la siguiente función:

In [206]:
def get_related_concepts(concept_ix, k, embed):
    W = embed.weight.data
    x = W[torch.tensor(concept_ix)]

    cos = torch.mv(W, x) / torch.sqrt(torch.sum(W * W, dim=1) *
                                      torch.sum(x * x) + 1e-9)
    topk = torch.topk(cos, k=k+1)[1].cpu().numpy().astype('int32')

    related = []
    for i in topk[1:]:
        related.append(vocabulary[i])
    return related

In [207]:
isinet = SkipGram(vocabulary, 256)
isinet.load_state_dict(torch.load(models_path + '/100-1-256-256.pt',weights_only=True))

<All keys matched successfully>

Algunos ejemplos de la misma serían:

In [208]:
# ('http', 'solicitud')
get_related_concepts(2400, 10, isinet.central_embedding)

[('navegador', 'web'),
 ('cliente', 'solicitud'),
 ('nombre', 'servidor'),
 ('envía', 'respuesta'),
 ('servidor', 'solicitud'),
 ('conexión', 'host'),
 ('cliente', 'mensaje'),
 ('respuesta', 'servidor'),
 ('cliente', 'envía'),
 ('cliente', 'web')]

In [209]:
# ('enviar', 'mensajes')
get_related_concepts(1475, 10, isinet.central_embedding)

[('archivo', 'programa'),
 ('archivo', 'descriptor'),
 ('actual', 'directorio'),
 ('descriptor', 'seguridad'),
 ('comandos', 'línea'),
 ('acceso', 'archivo'),
 ('/usr', 'ast'),
 ('archivos', 'programa'),
 ('archivo', 'contiene'),
 ('byte', 'bytes')]